In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("C:/Users/HP/10 Acadamy PRojects/New folder (5)/Altdata-lending-risk-model/data/data.csv")
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [6]:
# Group by CustomerId and compute aggregate features
agg_df = df.groupby("CustomerId").agg({
    "Amount": ["sum", "mean", "count", "std"],
    "Value": ["sum", "mean", "std"]
})

# Flatten column names
agg_df.columns = ["amount_sum", "amount_mean", "amount_count", "amount_std",
                  "value_sum", "value_mean", "value_std"]

# Reset index to make CustomerId a column again
agg_df = agg_df.reset_index()

agg_df.head()


,CustomerId,amount_sum,amount_mean,amount_count,amount_std,value_sum,value_mean,value_std
0,CustomerId_1,-10000.0,-10000.000000,1,NaN,10000,10000.000000,NaN
1,CustomerId_10,-10000.0,-10000.000000,1,NaN,10000,10000.000000,NaN
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333,30400,6080.000000,4100.243895
3,CustomerId_1002,4225.0,384.090909,11,560.498966,4775,434.090909,518.805446
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146,32000,5333.333333,3945.461528


In [7]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def create_aggregate_features(df):
    agg_df = df.groupby('CustomerId').agg({
        'Amount': ['sum', 'mean', 'std', 'count'],
        'TransactionId': 'nunique',  # Unique transactions
        'ProductCategory': 'nunique'  # Variety of products
    }).reset_index()
    agg_df.columns = ['CustomerId', 'total_amount', 'avg_amount', 'std_amount', 'transaction_count', 'unique_products']
    return agg_df

In [8]:
def extract_time_features(df):
    df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
    df['transaction_hour'] = df['TransactionStartTime'].dt.hour
    df['transaction_day'] = df['TransactionStartTime'].dt.day
    df['transaction_month'] = df['TransactionStartTime'].dt.month
    df['transaction_year'] = df['TransactionStartTime'].dt.year
    df['transaction_dayofweek'] = df['TransactionStartTime'].dt.dayofweek
    return df

In [9]:
def time_since_last_transaction(df):
    df = df.sort_values(['CustomerId', 'TransactionStartTime'])
    df['time_since_last'] = df.groupby('CustomerId')['TransactionStartTime'].diff().dt.total_seconds() / 3600
    df['time_since_last'] = df['time_since_last'].fillna(df['time_since_last'].max())
    return df

In [10]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def encode_categorical(df):
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
    cat_features = ['ProductCategory', 'ChannelId']
    ohe_df = pd.DataFrame(ohe.fit_transform(df[cat_features]), columns=ohe.get_feature_names_out())
    df = pd.concat([df.drop(cat_features, axis=1), ohe_df], axis=1)
    
    le = LabelEncoder()
    df['ProviderId'] = le.fit_transform(df['ProviderId'])
    return df

In [13]:
from sklearn.impute import SimpleImputer

def impute_missing(df):
    num_imputer = SimpleImputer(strategy='median')
    cat_imputer = SimpleImputer(strategy='most_frequent')
    numerical_cols = df.select_dtypes(include='number').columns
    categorical_cols = df.select_dtypes(include='object').columns
    df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])
    df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])
    return df

In [16]:
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df[numerical_cols] = knn_imputer.fit_transform(df[numerical_cols])

NameError: name 'numerical_cols' is not defined

In [17]:
from sklearn.impute import KNNImputer

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns  # auto select numerical columns

knn_imputer = KNNImputer(n_neighbors=5)
df[numerical_cols] = knn_imputer.fit_transform(df[numerical_cols])


In [18]:
def standardize_features(df):
    scaler = StandardScaler()
    numerical_cols = df.select_dtypes(include='number').columns
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    return df

In [19]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('aggregate_features', create_aggregate_features),
    ('time_features', extract_time_features),
    ('encode_categorical', encode_categorical),
    ('impute_missing', impute_missing),
    ('standardize', standardize_features)
])

In [20]:
from xverse.transformer import WOE
woe = WOE()
woe.fit(processed_df, processed_df['FraudResult'])
print(woe.iv_df)

NameError: name 'processed_df' is not defined